In [26]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

from time import time
import numpy as np
import pandas as pd

#vim ~/.Plotly/.Credentials

## Load Data

In [2]:
pgd = ph.phyloData()

the data of phylo


Pick one of the methods below for getting data

### 1) fetch from Tree of Life

In [3]:
pgd.search_name("primates")

,name,id
0,Primates,15963


In [4]:
#pgd.fetch_tol_data(15963, limit=25000)

In [5]:
#pgd.add_time()
#pgd.df.head()

### 2) load from raw file

In [6]:
#raw_file = "data/primate_raw.xml"
#pgd.load_raw_file(raw_file)

In [7]:
#pgd.write_prep_data("data/mammal_df.csv", "data/mammal_links.txt")

### 3) load from prepared files

In [8]:
#pgd.load_prep_data("data/Mammalia-15040-df.csv")
pgd.load_prep_data("data/Chordata-2499-df.csv")


### return data for plotting

In [9]:
df = pgd.return_data()
print(df.shape)
df.head()

(10395, 11)


,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End
0,2466,1,0,2499,Chordata,2,0,-0.794184,0.918448,521.0,0.0
1,2499,2,2,14819,Calcichordata,0,0,-0.091174,0.033661,NaN,NaN
2,2499,2,0,14821,Urochordata,8,0,-1.014761,0.366714,541.0,485.4
3,14821,3,0,114489,Octacnemidae,0,0,-0.066112,-0.370996,NaN,NaN
4,14821,3,0,114490,Phlebobranchiata,4,0,-0.344893,-0.201929,NaN,NaN


## Plot Data

In [10]:
pgp = ph.phyloGraph(df)

the plotting of phylo


In [12]:
pgp.search_name('Verte')

                        name     id
38                Vertebrata  14829
135  Terrestrial Vertebrates  14952


In [13]:
root_node=14829 # Vertebrata
#root_node=15040 # mammals
#root_node=15963 # primates

In [14]:
start = time()
pgp.create_plot_df(root_node)
print("create_plot_df() took {} secs".format(np.round(time() - start)))

/Users/Seth/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/Seth/Documents/tolGraph/phyloGraph.py:487: RuntimeWarning:

All-NaN slice encountered



create_plot_df() took 6065.0 secs


/Users/Seth/Documents/tolGraph/phyloGraph.py:541: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [15]:
start = time()
pgp.rejitter_XY()
print("rejitter_XY() took {} secs".format(np.round(time() - start)))

rejitter_XY() took 78.0 secs


In [23]:
#pgp.plot_df.to_csv("data/plot_df-Vertebrata.csv", index=False)

In [16]:
start = time()
pgp.create_plot_data()
#pgp.create_plot_data(Z_dim='depth', Z_dim_mult=-1)
print("create_plot_data() took {} secs".format(np.round(time() - start)))

Loaded plot data. Root node: 14829
create_plot_data() took 6.0 secs


In [17]:
#pgp.render_plot()

In [41]:
pgp.search_name('Antenna')

                                  name     id
10261                   Antennarioidei  21991
10263                    Antennariidae  21993
10264                      Antennarius  52009
10265       Antennarius striatus Group  52020
10266             Antennarius striatus  52046
10267             Antennarius hispidus  52047
10268       Antennarius nummifer Group  52026
10269             Antennarius nummifer  52048
10270            Antennarius coccineus  52051
10271          Antennarius bermudensis  52052
10272           Antennarius sanguineus  52053
10273              Antennarius duescus  52055
10274               Antennarius analis  52056
10275            Antennarius dorhensis  52057
10276             Antennarius rosaceus  52058
10277          Antennarius biocellatus  52028
10278  Antennarius pauciradiatus Group  52029
10279             Antennarius randalli  52065
10280        Antennarius pauciradiatus  52066
10281         Antennarius pictus Group  52022
10282               Antennarius pi

In [42]:
#focus_node = 16024 # whales
#focus_node = 16293 # Catarrhini
#focus_node = 16299 # Hominidae
pgp.refocus(21993)

/Users/Seth/Documents/tolGraph/phyloGraph.py:564: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Loaded plot data. Highlighting: 21993


In [20]:
#pgp.render_plot(publish=True, filename="Primates")

In [20]:
6065.0/3600

1.6847222222222222

In [24]:
pgp2 = ph.phyloGraph(df)

the plotting of phylo


In [28]:
pgp2.plot_df = pd.DataFrame.from_csv("data/plot_df-Vertebrata.csv")

In [29]:
pgp2.create_plot_data()

KeyError: 'ancestor'